In [134]:
# ----------------------------------------------------------------------------------- #
# SON LOS IMPORTS DEL LAB ANTERIOR, LOS DESCOMENTAMOS A MEDIDA QUE LOS VAYAMOS USANDO #
# ----------------------------------------------------------------------------------- #

# from collections import defaultdict
from array import array
import nltk
nltk.download('stopwords')
# # nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
import math
import numpy as np
import collections
from numpy import linalg as la
# import json
# import random
# import re
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tonij\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading the processed tweets

In [135]:
output_folder = '../output/'
data = pd.read_csv(output_folder + "lab1_tweets_df.csv", sep='|')
data.head()

,Document,Id,Text,Username,Date,Hashtags,Mentions,Likes,Retweets,Url,Clean_text
0,doc_1,1575918182698979328,So this will keep spinning over us until 7 pm…...,suzjdean,Fri Sep 30 18:39:08 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/suzjdean/status/1575918182...,"['keep', 'spin', 'us', 'away', 'alreadi', 'hur..."
1,doc_2,1575918151862304768,Our hearts go out to all those affected by #Hu...,lytx,Fri Sep 30 18:39:01 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/lytx/status/15759181518623...,"['heart', 'go', 'affect', 'hurricaneian', 'wis..."
2,doc_3,1575918140839673873,Kissimmee neighborhood off of Michigan Ave. \r...,CHeathWFTV,Fri Sep 30 18:38:58 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/CHeathWFTV/status/15759181...,"['kissimme', 'neighborhood', 'michigan', 'hurr..."
3,doc_4,1575918135009738752,I have this one tree in my backyard that scare...,spiralgypsy,Fri Sep 30 18:38:57 +0000 2022,"['scwx', 'HurricaneIan']",NaN,0,0,https://twitter.com/spiralgypsy/status/1575918...,"['one', 'tree', 'backyard', 'scare', 'polterge..."
4,doc_5,1575918119251419136,@AshleyRuizWx @Stephan89441722 @lilmizzheidi @...,Blondie610,Fri Sep 30 18:38:53 +0000 2022,['HurricaneIan'],"['AshleyRuizWx', 'Stephan89441722', 'lilmizzhe...",0,0,https://twitter.com/Blondie610/status/15759181...,"['ashleyruizwx', 'lilmizzheidi', 'winknew', 'd..."


# Indexing

### 1. Build inverted index

In [136]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line=  line.lower() ## Transform in lowercase
    line=  line.split() ## Tokenize the text to get a list of terms
    line= [x for x in line if x not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemmer.stem(x) for x in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [148]:
def create_index_tfidf(dataframe):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    num_documents -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    num_documents = dataframe.shape[0]
    index = defaultdict(list)
    tf = defaultdict(list)  # term frequencies of terms in documents (documents in the same order as in the main index)
    df = defaultdict(int)  # document frequencies of terms in the corpus
    author_index = defaultdict(str)
    idf = defaultdict(float)

    for row in dataframe.iterrows():
        tweet_id = row[1]['Id']
        terms = row[1]['Clean_text']  
        terms = eval(terms)
        username = row[1]['Username']
        author_index[tweet_id] = username

        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweet_id, [position]]
                
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        # calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] = df[term]+1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)

    return index, tf, df, idf, author_index


In [149]:
start_time = time.time()
new_index, tf, df, idf, author_index = create_index_tfidf(data)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 164.72 seconds


In [ ]:
new_index

### 2. Propose test queries

The proposed queries that we are going to use are:

- Donald Trump
- Joe Biden
- Covid 19
- Data Science
- Information Retrieval

### 3. Rank your results

First we are going to implement the `tf-idf algorithm` with our inverted index, then we are going to rank the results of the proposed queries.

In [150]:
def rank_documents(terms, docs, index, idf, tf, author_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    author_index -- mapping between page id and author username
    
    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]=query_terms_count[term]/query_norm *idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf
    
    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)
    #print ('\n'.join(result_docs), '\n')
    return result_docs

In [151]:
def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, title_index)
    return ranked_docs

In [ ]:
data.loc[]

In [161]:
print("Insert your query (i.e.: Computer Science):\n")
queries = ["Donald Trump", "Joe Biden", "Covid Vaccine", "Data Science", "Information retrieval"]
for query in queries:
    ranked_docs = search_tf_idf(query, new_index)
    top = 10

    print("\n======================\nTop {} results out of {} for the searched query '{}':\n".format(top, len(ranked_docs),query))
    for d_id in ranked_docs[:top]:
        print("tweet_id= {} - username: {}".format(d_id, author_index[d_id]))

Insert your query (i.e.: Computer Science):


Top 10 results out of 16 for the searched query 'Donald Trump':

tweet_id= 1575916233295532032 - username: A_NHDogLover
tweet_id= 1575858998116769792 - username: LossGu
tweet_id= 1575864549089357824 - username: Tkarera
tweet_id= 1575910806146801665 - username: Grt8T
tweet_id= 1575868670580260864 - username: SilentTiger777
tweet_id= 1575906173366112256 - username: two_thirty_two
tweet_id= 1575910393767133184 - username: realgjohnson5
tweet_id= 1575862784062984192 - username: RepubliGAY
tweet_id= 1575883089141981186 - username: DennisMy24
tweet_id= 1575905023477051392 - username: GeronimoBPM

Top 10 results out of 71 for the searched query 'Joe Biden':

tweet_id= 1575880523817242624 - username: NBC10
tweet_id= 1575863552233320448 - username: GeorgeArztComm
tweet_id= 1575909119604785153 - username: cgtnamerica
tweet_id= 1575879754212954112 - username: NewsNation
tweet_id= 1575870584398962692 - username: IrishMirror
tweet_id= 157590554718170726

# Evaluation